In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input

Using TensorFlow backend.


In [2]:
TOT = 49673580
EPOCHS = 50
size = 128
batchsize = 200
lw = 6
channel = 1
STEPS = TOT / EPOCHS / batchsize / 3
NCATS = 340

In [3]:
K.clear_session()
# base_model = MobileNet(input_shape=(size, size, 3), alpha=1., weights='imagenet', include_top=False)
# x = base_model.output
# # x = GlobalAveragePooling2D()(x)
# x = Flatten()(x)
# x = Dropout(0.5)(x)

# x = Dense(512)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

# x = Dense(64)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

# predictions = Dense(NCATS, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=predictions)

model = MobileNet(input_shape=(size, size, 1), alpha=1., weights=None, classes=NCATS)

model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
__________

In [4]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, preprocess_input=preprocess_input, channel=channel)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

In [5]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 128, 128, 1) (34000, 340)
Validation array memory 2.08 GB


In [6]:
train_datagen = image_generator_xd(size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                  channel=channel)

In [7]:
model_prefix = 'mobilenet_raw_head_{}_lw{}'.format(size, lw)
fold = 9
callbks = [
    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
                      min_delta=0.005, mode='max', cooldown=3, verbose=1),
    callbacks.ModelCheckpoint("./models/{}.model".format(model_prefix),
                                monitor='val_categorical_accuracy', 
                                mode = 'max', save_best_only=True, verbose=1),
    callbacks.TensorBoard(log_dir='./log/{}'.format(model_prefix)),
]

In [8]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

Epoch 1/50
1656/1655 [==============================] - 464s 280ms/step - loss: 2.3993 - categorical_crossentropy: 2.3993 - categorical_accuracy: 0.4520 - top_3_accuracy: 0.6415 - val_loss: 2.2562 - val_categorical_crossentropy: 2.2562 - val_categorical_accuracy: 0.4790 - val_top_3_accuracy: 0.6858

Epoch 00001: val_categorical_accuracy improved from -inf to 0.47900, saving model to ./models/mobilenet_raw_head_128_lw6.model
Epoch 2/50
1656/1655 [==============================] - 472s 285ms/step - loss: 1.4754 - categorical_crossentropy: 1.4754 - categorical_accuracy: 0.6330 - top_3_accuracy: 0.8201 - val_loss: 4.4436 - val_categorical_crossentropy: 4.4436 - val_categorical_accuracy: 0.1896 - val_top_3_accuracy: 0.3356

Epoch 00002: val_categorical_accuracy did not improve from 0.47900
Epoch 3/50
1656/1655 [==============================] - 473s 286ms/step - loss: 1.3099 - categorical_crossentropy: 1.3099 - categorical_accuracy: 0.6707 - top_3_accuracy: 0.8485 - val_loss: 1.3819 - val_c

Epoch 21/50
1656/1655 [==============================] - 473s 286ms/step - loss: 0.9069 - categorical_crossentropy: 0.9069 - categorical_accuracy: 0.7670 - top_3_accuracy: 0.9071 - val_loss: 0.9284 - val_categorical_crossentropy: 0.9284 - val_categorical_accuracy: 0.7637 - val_top_3_accuracy: 0.9030

Epoch 00021: val_categorical_accuracy did not improve from 0.76615
Epoch 22/50
1656/1655 [==============================] - 473s 286ms/step - loss: 0.9048 - categorical_crossentropy: 0.9048 - categorical_accuracy: 0.7674 - top_3_accuracy: 0.9075 - val_loss: 0.9423 - val_categorical_crossentropy: 0.9423 - val_categorical_accuracy: 0.7555 - val_top_3_accuracy: 0.9015

Epoch 00022: val_categorical_accuracy did not improve from 0.76615
Epoch 23/50
1656/1655 [==============================] - 472s 285ms/step - loss: 0.9053 - categorical_crossentropy: 0.9053 - categorical_accuracy: 0.7665 - top_3_accuracy: 0.9069 - val_loss: 0.9167 - val_categorical_crossentropy: 0.9167 - val_categorical_accurac

1656/1655 [==============================] - 467s 282ms/step - loss: 0.7885 - categorical_crossentropy: 0.7885 - categorical_accuracy: 0.7972 - top_3_accuracy: 0.9219 - val_loss: 0.7803 - val_categorical_crossentropy: 0.7803 - val_categorical_accuracy: 0.7989 - val_top_3_accuracy: 0.9233

Epoch 00041: val_categorical_accuracy did not improve from 0.79932
Epoch 42/50
1656/1655 [==============================] - 467s 282ms/step - loss: 0.7860 - categorical_crossentropy: 0.7860 - categorical_accuracy: 0.7975 - top_3_accuracy: 0.9226 - val_loss: 0.7714 - val_categorical_crossentropy: 0.7714 - val_categorical_accuracy: 0.8019 - val_top_3_accuracy: 0.9249

Epoch 00042: val_categorical_accuracy improved from 0.79932 to 0.80191, saving model to ./models/mobilenet_raw_head_128_lw6.model
Epoch 43/50
1656/1655 [==============================] - 467s 282ms/step - loss: 0.7884 - categorical_crossentropy: 0.7884 - categorical_accuracy: 0.7970 - top_3_accuracy: 0.9223 - val_loss: 0.7746 - val_categor

In [9]:
model.load_weights('./models/{}.model'.format(model_prefix))

In [10]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input,
                             channel=channel)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

(112199, 3) (112199, 128, 128, 1)
Test array memory 6.85 GB


In [11]:
test_predictions = model.predict(x_test, batch_size=128, verbose=1)

top3 = preds2catids(test_predictions)
top3.head()
top3.shape

112199/112199 [==============================] - 34s 299us/step


(112199, 3)

In [12]:
np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

(112199, 3)

In [13]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.head()

,key_id,word
0,9000003627287624,radio stereo snorkel
1,9000010688666847,hockey_puck bottlecap pool
2,9000023642890129,The_Great_Wall_of_China castle fence
3,9000038588854897,mountain The_Eiffel_Tower tent
4,9000052667981386,fireplace campfire fire_hydrant


In [14]:
import kaggle_util
kaggle_util.save_result(submission, 
                        '../result/{}.csv'.format(model_prefix), 
                        'quickdraw-doodle-recognition', 
                        send=True, index=False)

save result
upload result
cmd: kaggle competitions submit -c quickdraw-doodle-recognition -f ../result/mobilenet_raw_head_128_lw6.csv.7z -m "submit"
